# **Bonus Project - Data Analytics for Finance**
### 2023 - S2


<br>

Student Name | Student Number
--- | ---
Miguel Teodoro | 53127

<br>

**Idea:** Combine Signals from Idiosyncratic Skewness and Volatilty from the Fama-French 3-factor model

<br>


In [1]:
// Import the necessary packages/libraries

#r "nuget: FsUnit.Xunit"
#r "nuget: xunit, 2.*"
#r "nuget:FSharp.Data,5.*"
#r "nuget: FSharp.Stats"
#r "nuget: DiffSharp-lite"
#r "nuget: NovaSBE.Finance, 0.2.0-beta1"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: Plotly.NET, 3.*"
#r "nuget: Plotly.NET.Interactive, 3.*"
#r "nuget: Quotes.YahooFinance, 0.0.5"

open Xunit
open FsUnit.Xunit
open FsUnitTyped
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats
open MathNet.Numerics.Statistics
open NovaSBE.Finance.Ols
open Quotes.YahooFinance
open DiffSharp

Installed Packages DiffSharp-lite, 1.0.7 FSharp.Data, 5.0.2 FSharp.Stats, 0.4.11 FsUnit.Xunit, 5.3.0 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NovaSBE.Finance, 0.2.0-beta1 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2 Quotes.YahooFinance, 0.0.5 xunit, 2.4.2

Loading extensions from `C:\Users\migue\.nuget\packages\plotly.net.interactive\3.0.2\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

Import files. In this case we will start by performing the same evaluation for idiosyncratic volatility, as the one that was made for idiosyncratic skewness in the Final Project.

In [2]:
open NovaSBE.Finance.Portfolio

let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

let [<Literal>] IdAndReturnsFilePath = "data/id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "data/ivol_ff3_21d.csv"
let strategyName = "ivol21"

In [3]:
let idAndReturnsCsv = 
    CsvProvider<IdAndReturnsFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList
    
let mySignalCsv = 
    CsvProvider<MySignalFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList

In [4]:
// store signals
let mySignals =
    mySignalCsv
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Signal = signal * (-1.0)}
            Some signalRecord)

In [5]:
// store returns
let myReturns =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.Ret with
        | None -> None
        | Some ret ->
            let ret: SecurityReturn =
                { SecurityId = Other row.Id
                  Date = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Return= ret }
            Some ret)

In [6]:
// store market capitalizations
let myMktCaps =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.MarketEquity with
        | None -> None
        | Some mktCap ->
            let mktCap: WeightVariable =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Value = mktCap }
            Some mktCap)

### **1.** Evaluating Idiosyncratic Volatility

In [7]:
// create a value-weighted backtest for the signal
let backtest = Backtest(returns=myReturns, signals=mySignals, nPortfolios=3, name = strategyName)
let vw = backtest.strategyValueWeighted(myMktCaps)

In [8]:
open NovaSBE.Finance

// get rf data
let ff3Lookup = 
    French.getFF3 French.Frequency.Monthly
    |> Array.map (fun x -> DateTime(x.Date.Year, x.Date.Month, 1), x)
    |> Map

// apply it to calculate excess returns
let vwExcessReturns =
    vw.Returns
    |> List.map(fun x -> 
        { Name = x.Name
          Index = x.Index
          Month = x.Month
          Return = x.Return - ff3Lookup[x.Month].Rf})

In [9]:
// get the long portfolio
let long =
    vwExcessReturns
    |> List.filter (fun row -> row.Index = 3)
    
// get the short portfolio
let short =
    vwExcessReturns
    |> List.filter (fun row -> row.Index = 1)

// get the second tercile portfolio (won't be used)
// let medium =
//     vwExcessReturns
//     |> List.filter (fun row -> row.Index = 2)

// create long-short portfolio
let longShort =
    vw.Returns
    |> List.groupBy (fun x -> x.Month)
    |> List.map (fun (month, xs) ->
        let long = xs |> List.find (fun x -> x.Index = 3)
        let short = xs |> List.find (fun x -> x.Index = 1)
        { Name = "Long-Short"
          Index = 0
          Month = long.Month
          Return = long.Return - short.Return})

// get the value-weighted stock market excess returns portfolio (fama-french data comes as value-weighted)
let market =
    // impose limit dates to compare with portfolios
    let maxDate = vwExcessReturns |> List.map(fun x -> x.Month) |> List.max
    let minDate = vwExcessReturns |> List.map(fun x -> x.Month) |> List.min
    French.getFF3 French.Frequency.Monthly
    |> Array.toList
    |> List.filter(fun x -> x.Date <= maxDate)
    |> List.filter(fun x -> x.Date >= minDate)
    |> List.map(fun x ->
        { Name = "Market"
          Index = 10
          Month = x.Date
          Return = x.MktRf })

In [10]:
// function to calculate cumulative returns
let cumulativeReturn (xs: PortfolioReturn list) =
    let mutable cr = 1.0
    let sorted = xs |> List.sortBy(fun x -> x.Month)
    [ for x in sorted do 
        cr <- cr * (1.0 + x.Return)
        { x with Return = cr } ]

In [11]:
// function to plot cumulative returns
let plotCumulativeRets (xs: PortfolioReturn list) =
    xs
    |> cumulativeReturn
    |> List.map(fun x -> x.Month, x.Return)
    |> Chart.Line 
    |> Chart.withLegendStyle(Orientation=StyleParam.Orientation.Horizontal)

// creates chart and shows
let combinedChart =
    [plotCumulativeRets long |> Chart.withTraceInfo(Name="Long Portfolio");
     plotCumulativeRets short |> Chart.withTraceInfo(Name="Short Portfolio");
     plotCumulativeRets market |> Chart.withTraceInfo(Name="Market Excess Returns");
     plotCumulativeRets longShort |> Chart.withTraceInfo(Name="Long-Short Portfolio");]
    |> Chart.combine
    |> Chart.withTitle "Cumulative Returns for Value-Weighted Portfolios"

combinedChart

<!-- Plotly chart will be drawn inside this DIV -->

In [12]:
// function calculate annual standard deviations from monthly
let annualizeMonthlyStdDev monthlyStdDev = sqrt(12.0) * monthlyStdDev

// function to annualize portfolios standard deviations
let stdDevPortfolioAnnualized (xs: PortfolioReturn list) =
    xs
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

// function to leverage portfolio to have 10% volatility
let vol10 (xs: PortfolioReturn list) =
    xs
    |> List.map(fun x ->
        let w = 0.1 / stdDevPortfolioAnnualized xs
        { Name = x.Name
          Index = x.Index
          Month = x.Month
          Return = x.Return * w
        })

// plots leveraged portfolios
let combinedChartVol =
    [plotCumulativeRets (vol10 long) |> Chart.withTraceInfo(Name="Long Portfolio");
     plotCumulativeRets (vol10 short) |> Chart.withTraceInfo(Name="Short Portfolio");
     plotCumulativeRets (vol10 market) |> Chart.withTraceInfo(Name="Market Excess Returns");
     plotCumulativeRets (vol10 longShort) |> Chart.withTraceInfo(Name="Long-Short Portfolio");]
    |> Chart.combine
    |> Chart.withTitle "Cumulative Returns for Value-Weighted Portfolios with Constant Leverage"

combinedChartVol

<!-- Plotly chart will be drawn inside this DIV -->

In [13]:
// splits the data in two halves
let splitInTwo (xs: PortfolioReturn list) =
    xs
    |> List.sortBy (fun x -> x.Month)
    |> List.splitInto 2


In [14]:
// applies the splitting function and stores the halves

let long1 = (splitInTwo long)[0]
let long2 = (splitInTwo long)[1]

let longShort1 = (splitInTwo longShort)[0]
let longShort2 = (splitInTwo longShort)[1]

let market1 = (splitInTwo market)[0]
let market2 = (splitInTwo market)[1]

In [15]:
// function to calculate annualized returns from monthly returns
let annualizeMonthlyReturns monthlyReturns = 12.0 * monthlyReturns

// function to annualize portfolios average returns
let returnsPortfolioAnnualized (xs: PortfolioReturn list) =
    xs
    |> Seq.averageBy (fun x -> x.Return)
    |> annualizeMonthlyReturns

In [16]:
// for each portfolio stores annualized returns

let returnLong = returnsPortfolioAnnualized long
let returnLong1 = returnsPortfolioAnnualized long1
let returnLong2 = returnsPortfolioAnnualized long2

let returnLongShort = returnsPortfolioAnnualized longShort
let returnLongShort1 = returnsPortfolioAnnualized longShort1
let returnLongShort2 = returnsPortfolioAnnualized longShort2

let returnMarket = returnsPortfolioAnnualized market
let returnMarket1 = returnsPortfolioAnnualized market1
let returnMarket2 = returnsPortfolioAnnualized market2

In [17]:
// for each portfolio stores annualized volatility

let stdDevLong = stdDevPortfolioAnnualized long
let stdDevLong1 = stdDevPortfolioAnnualized long1
let stdDevLong2 = stdDevPortfolioAnnualized long2

let stdDevLongShort = stdDevPortfolioAnnualized longShort
let stdDevLongShort1 = stdDevPortfolioAnnualized longShort1
let stdDevLongShort2 = stdDevPortfolioAnnualized longShort2

let stdDevMarket = stdDevPortfolioAnnualized market
let stdDevMarket1 = stdDevPortfolioAnnualized market1
let stdDevMarket2 = stdDevPortfolioAnnualized market2

In [18]:
// function to calculate portfolios sharpe ratio
let srPortfolio (xs: PortfolioReturn list) = 
    (returnsPortfolioAnnualized xs) / (stdDevPortfolioAnnualized xs)

In [19]:
// for each portfolio stores sharpe ratio

let srLong = srPortfolio long
let srLong1 = srPortfolio long1
let srLong2 = srPortfolio long2

let srLongShort = srPortfolio longShort
let srLongShort1 = srPortfolio longShort1
let srLongShort2 = srPortfolio longShort2

let srMarket = srPortfolio market
let srMarket1 = srPortfolio market1
let srMarket2 = srPortfolio market2

In [20]:
// creates regression data type
type RegData =
     {  Date: DateTime
        Portfolio: float
        MktRf: float
        Hml: float
        Smb: float }

// function to adapt portfolio returns to regression data type
let adaptRD (xs: PortfolioReturn list) =
    xs
    |> List.map (fun x ->
        let xff3 = ff3Lookup[x.Month]
        { Date = x.Month
          Portfolio = x.Return
          MktRf = xff3.MktRf
          Hml = xff3.Hml
          Smb = xff3.Smb })

In [21]:
// functions to fit CAPM and Fama-French-3 factor models

let fitCAPM (xs: RegData list) =
    Ols("Portfolio ~ MktRf", xs).fit()

let fitFF3 (xs: RegData list) =
    Ols("Portfolio ~ MktRf + Hml + Smb", xs).fit()

In [22]:
// creates a type to suit regression outputs
type outputReg =
    { Alpha_CAPM: float
      Alpha_TstatCAPM: float
      MktRf_CoefCAPM: float
      MktRf_TstatCAPM: float
      R2_CAPM: float
      IR_CAPM: float

      Alpha_FF3: float
      Alpha_TstatFF3: float
      MktRf_CoefFF3: float
      MktRf_TstatFF3: float
      Hml_CoefFF3: float
      Hml_TstatFF3: float
      Smb_CoefFF3: float
      Smb_TstatFF3: float
      R2_FF3: float
      IR_FF3: float
      }

In [23]:
// function that fits CAPM and FF3 regressions to a portfolio and stores the models outputs
let fitRegs(xs: RegData list) =

    let capm = fitCAPM(xs)
    let ff3 = fitFF3(xs)

    // calculate IR
    let capmStDevResiduals = (Seq.stDev capm.resid)
    let capmIR = capm.coefs["Intercept"] / capmStDevResiduals

    let ff3StDevResiduals = (Seq.stDev ff3.resid)
    let ff3IR = ff3.coefs["Intercept"] / ff3StDevResiduals

    { Alpha_CAPM = capm.coefs["Intercept"]
      Alpha_TstatCAPM = capm.tvalues["Intercept"]
      MktRf_CoefCAPM = capm.coefs["MktRf"]
      MktRf_TstatCAPM = capm.tvalues["MktRf"]
      R2_CAPM = capm.rsquared
      IR_CAPM = capmIR

      Alpha_FF3 = ff3.coefs["Intercept"]
      Alpha_TstatFF3 = ff3.tvalues["Intercept"]
      MktRf_CoefFF3 = ff3.coefs["MktRf"]
      MktRf_TstatFF3 = ff3.tvalues["MktRf"]
      Hml_CoefFF3 = ff3.coefs["Hml"]
      Hml_TstatFF3 = ff3.tvalues["Hml"]
      Smb_CoefFF3 = ff3.coefs["Smb"]
      Smb_TstatFF3 = ff3.tvalues["Smb"]
      R2_FF3 = ff3.rsquared
      IR_FF3 = ff3IR
    }

In [24]:
// regressions of both the Long-Only and the Long-Short portfolios

let regLong = fitRegs (adaptRD long)
let regLong1 = fitRegs (adaptRD long1)
let regLong2 = fitRegs (adaptRD long2)

let regLongShort = fitRegs (adaptRD longShort)
let regLongShort1 = fitRegs (adaptRD longShort1)
let regLongShort2 = fitRegs (adaptRD longShort2)

In [25]:
// eases visibility in following table
let roundDisplay (value: float) = Math.Round(value, 3).ToDisplayString()

In [26]:
// creates table with portfolio statistics

let header = 
    [ 
    "<b>Portfolio"      ; "<b>Avg. Excess Return"       ; "<b>Avg. Volatility"          ; "<b>Sharpe Ratio"         ; "<b>CAPM Alpha"                          ; "<b>CAPM Alpha t-stat"                        ; "<b>CAPM R-squared"                   ; "<b>CAPM IR"                          ; "<b>FF3 Alpha"                        ; "<b>FF3 Alpha t-stat"                     ; "<b>FF3 R-squared"                ; "<b>FF3 IR"                       
    ]

let rows = 
    [
     ["Long-only"       ; roundDisplay returnLong       ; roundDisplay stdDevLong       ; roundDisplay srLong       ; roundDisplay regLong.Alpha_CAPM          ; roundDisplay regLong.Alpha_TstatCAPM          ; roundDisplay regLong.R2_CAPM          ; roundDisplay regLong.IR_CAPM          ; roundDisplay regLong.Alpha_FF3        ; roundDisplay regLong.Alpha_TstatFF3       ; roundDisplay regLong.R2_FF3       ; roundDisplay regLong.IR_FF3       ]           
     ["Long-only 1st"   ; roundDisplay returnLong1      ; roundDisplay stdDevLong1      ; roundDisplay srLong1      ; roundDisplay regLong1.Alpha_CAPM         ; roundDisplay regLong1.Alpha_TstatCAPM         ; roundDisplay regLong1.R2_CAPM         ; roundDisplay regLong1.IR_CAPM         ; roundDisplay regLong1.Alpha_FF3       ; roundDisplay regLong1.Alpha_TstatFF3      ; roundDisplay regLong1.R2_FF3      ; roundDisplay regLong1.IR_FF3      ]   
     ["Long-only 2nd"   ; roundDisplay returnLong2      ; roundDisplay stdDevLong2      ; roundDisplay srLong2      ; roundDisplay regLong2.Alpha_CAPM         ; roundDisplay regLong2.Alpha_TstatCAPM         ; roundDisplay regLong2.R2_CAPM         ; roundDisplay regLong2.IR_CAPM         ; roundDisplay regLong2.Alpha_FF3       ; roundDisplay regLong2.Alpha_TstatFF3      ; roundDisplay regLong2.R2_FF3      ; roundDisplay regLong2.IR_FF3      ]
     ["Long-Short"      ; roundDisplay returnLongShort  ; roundDisplay stdDevLongShort  ; roundDisplay srLongShort  ; roundDisplay regLongShort.Alpha_CAPM     ; roundDisplay regLongShort.Alpha_TstatCAPM     ; roundDisplay regLongShort.R2_CAPM     ; roundDisplay regLongShort.IR_CAPM     ; roundDisplay regLongShort.Alpha_FF3   ; roundDisplay regLongShort.Alpha_TstatFF3  ; roundDisplay regLongShort.R2_FF3  ; roundDisplay regLongShort.IR_FF3  ] 
     ["Long-Short 1st"  ; roundDisplay returnLongShort1 ; roundDisplay stdDevLongShort1 ; roundDisplay srLongShort1 ; roundDisplay regLongShort1.Alpha_CAPM    ; roundDisplay regLongShort1.Alpha_TstatCAPM    ; roundDisplay regLongShort1.R2_CAPM    ; roundDisplay regLongShort1.IR_CAPM    ; roundDisplay regLongShort1.Alpha_FF3  ; roundDisplay regLongShort1.Alpha_TstatFF3 ; roundDisplay regLongShort1.R2_FF3 ; roundDisplay regLongShort1.IR_FF3 ] 
     ["Long-Short 2nd"  ; roundDisplay returnLongShort2 ; roundDisplay stdDevLongShort2 ; roundDisplay srLongShort2 ; roundDisplay regLongShort2.Alpha_CAPM    ; roundDisplay regLongShort2.Alpha_TstatCAPM    ; roundDisplay regLongShort2.R2_CAPM    ; roundDisplay regLongShort2.IR_CAPM    ; roundDisplay regLongShort2.Alpha_FF3  ; roundDisplay regLongShort2.Alpha_TstatFF3 ; roundDisplay regLongShort2.R2_FF3 ; roundDisplay regLongShort2.IR_FF3 ]
     ["Market"          ; roundDisplay returnMarket     ; roundDisplay stdDevMarket     ; roundDisplay srMarket     ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
     ["Market 1st"      ; roundDisplay returnMarket1    ; roundDisplay stdDevMarket1    ; roundDisplay srMarket1    ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
     ["Market 2nd"      ; roundDisplay returnMarket2    ; roundDisplay stdDevMarket2    ; roundDisplay srMarket2    ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
    ]
 

Chart.Table(header, rows) 
|> Chart.withSize (1550, 450) 
|> Chart.withTitle("Performance Measures")

<!-- Plotly chart will be drawn inside this DIV -->

## **2.** Combine Idiosyncratic Volatility and Skewness

Import files.

In [27]:
let [<Literal>] MySignalFilePathSkew = "data/iskew_ff3_21d.csv"
let strategyNameSkew = "iskew21"

let [<Literal>] MySignalFilePathVol = "data/ivol_ff3_21d.csv"
let strategyNameVol = "ivol21"

In [28]:
let mySignalCsvSkew = 
    CsvProvider<MySignalFilePathSkew,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList

let mySignalCsvVol = 
    CsvProvider<MySignalFilePathVol,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList

In [29]:
// store skewness signal
let mySignalsSkew =
    mySignalCsvSkew
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Signal = signal }
            Some signalRecord)

// store volatility signal
let mySignalsVol =
    mySignalCsvVol
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Signal = signal * (-1.0)} 
            Some signalRecord)

In [30]:
// function to calculate daily average per signal
let getDailySignalAverage (signal: Signal list) =
    signal
    |> List.groupBy (fun x -> x.FormationDate)
    |> List.map (fun (group, securities) -> 
        let avg = securities |> List.map (fun x -> x.Signal) |> List.average 
        (group, avg))
    |> Map

// function to calculate daily standard deviation per signal
let getDailySignalStdDev (signal: Signal list) =
    signal
    |> List.groupBy (fun x -> x.FormationDate)
    |> List.map (fun (group, securities) -> 
        let stdev = securities |> List.map (fun x -> x.Signal) |> Seq.stDev 
        (group, stdev))
    |> Map

// function to normalize signal according to its daily average and std. dev.
let transformSignal (signal: Signal list) (signalDailyAverage: Map<DateTime,float>) (signalDailyStdDev: Map<DateTime,float>) =
    signal
    |> List.map(fun x ->
        { SecurityId = x.SecurityId
          FormationDate = x.FormationDate
          Signal = (x.Signal - signalDailyAverage[x.FormationDate]) / signalDailyStdDev[x.FormationDate]})


In [31]:
// applies normalization to skewness
let mySignalsSkewNormal =

    let dayAvg = getDailySignalAverage mySignalsSkew
    let dayStdDev = getDailySignalStdDev mySignalsSkew
    transformSignal mySignalsSkew dayAvg dayStdDev 

// applies normalization to volatility
let mySignalsVolNormal =

    let dayAvg = getDailySignalAverage mySignalsVol
    let dayStdDev = getDailySignalStdDev mySignalsVol
    transformSignal mySignalsVol dayAvg dayStdDev 

In [32]:
// gets a combined signal by summing both signals normalized
let sumNormalSignals (x: Signal list) (y: Signal list) =
    let xMap = 
        x
        |> List.map (fun x ->
            ((x.SecurityId, x.FormationDate), x.Signal))
        |> Map
    let yMap = 
        y
        |> List.map (fun y ->
            ((y.SecurityId, y.FormationDate), y.Signal))
        |> Map
    let overlappingObs =
        [ xMap.Keys |> set
          yMap.Keys |> set]
        |> Set.intersectMany
    [ for obs in overlappingObs do
        { SecurityId = fst obs
          FormationDate = snd obs
          Signal = xMap[obs] + yMap[obs]}]

// stores the normalized signal
let myNormalSignal = sumNormalSignals mySignalsSkewNormal mySignalsVolNormal

### **2.1.** Strategy Analysis

In [33]:
// create a value-weighted backtest for the signal
let backtest = Backtest(returns=myReturns, signals=myNormalSignal, nPortfolios=3, name = "iVolSkew")
let vw = backtest.strategyValueWeighted(myMktCaps)

In [34]:
// calculate excess returns
let vwExcessReturns =
    vw.Returns
    |> List.map(fun x -> 
        { Name = x.Name
          Index = x.Index
          Month = x.Month
          Return = x.Return - ff3Lookup[x.Month].Rf})

In [35]:
// get the long portfolio
let long =
    vwExcessReturns
    |> List.filter (fun row -> row.Index = 3)
    
// get the short portfolio
let short =
    vwExcessReturns
    |> List.filter (fun row -> row.Index = 1)

// get the second tercile portfolio (won't be used)
// let medium =
//     vwExcessReturns
//     |> List.filter (fun row -> row.Index = 2)

// create long-short portfolio
let longShort =
    vw.Returns
    |> List.groupBy (fun x -> x.Month)
    |> List.map (fun (month, xs) ->
        let long = xs |> List.find (fun x -> x.Index = 3)
        let short = xs |> List.find (fun x -> x.Index = 1)
        { Name = "Long-Short"
          Index = 0
          Month = long.Month
          Return = long.Return - short.Return})         //  - ff3Lookup[long.Month].Rf

// get the value-weighted stock market excess returns portfolio (fama-french data comes as value-weighted)
let market =
    // impose limit dates to compare with portfolios
    let maxDate = vwExcessReturns |> List.map(fun x -> x.Month) |> List.max
    let minDate = vwExcessReturns |> List.map(fun x -> x.Month) |> List.min
    French.getFF3 French.Frequency.Monthly
    |> Array.toList
    |> List.filter(fun x -> x.Date <= maxDate)
    |> List.filter(fun x -> x.Date >= minDate)
    |> List.map(fun x ->
        { Name = "Market"
          Index = 10
          Month = x.Date
          Return = x.MktRf })

In [36]:
let combinedChart =
    [plotCumulativeRets long |> Chart.withTraceInfo(Name="Long Portfolio");
     plotCumulativeRets short |> Chart.withTraceInfo(Name="Short Portfolio");
     plotCumulativeRets market |> Chart.withTraceInfo(Name="Market Excess Returns");
     plotCumulativeRets longShort |> Chart.withTraceInfo(Name="Long-Short Portfolio");]
    |> Chart.combine
    |> Chart.withTitle "Cumulative Returns for Value-Weighted Portfolios"

combinedChart

<!-- Plotly chart will be drawn inside this DIV -->

In [37]:
// plots leveraged portfolios
let combinedChartVol =
    [plotCumulativeRets (vol10 long) |> Chart.withTraceInfo(Name="Long Portfolio");
     plotCumulativeRets (vol10 short) |> Chart.withTraceInfo(Name="Short Portfolio");
     plotCumulativeRets (vol10 market) |> Chart.withTraceInfo(Name="Market Excess Returns");
     plotCumulativeRets (vol10 longShort) |> Chart.withTraceInfo(Name="Long-Short Portfolio");]
    |> Chart.combine
    |> Chart.withTitle "Cumulative Returns for Value-Weighted Portfolios with Constant Leverage"

combinedChartVol

<!-- Plotly chart will be drawn inside this DIV -->

In [38]:
// applies the splitting function and stores the halves

let long1 = (splitInTwo long)[0]
let long2 = (splitInTwo long)[1]

let longShort1 = (splitInTwo longShort)[0]
let longShort2 = (splitInTwo longShort)[1]

let market1 = (splitInTwo market)[0]
let market2 = (splitInTwo market)[1]

In [39]:
// for each portfolio stores annualized returns

let returnLong = returnsPortfolioAnnualized long
let returnLong1 = returnsPortfolioAnnualized long1
let returnLong2 = returnsPortfolioAnnualized long2

let returnLongShort = returnsPortfolioAnnualized longShort
let returnLongShort1 = returnsPortfolioAnnualized longShort1
let returnLongShort2 = returnsPortfolioAnnualized longShort2

let returnMarket = returnsPortfolioAnnualized market
let returnMarket1 = returnsPortfolioAnnualized market1
let returnMarket2 = returnsPortfolioAnnualized market2

In [40]:
// for each portfolio stores annualized volatility

let stdDevLong = stdDevPortfolioAnnualized long
let stdDevLong1 = stdDevPortfolioAnnualized long1
let stdDevLong2 = stdDevPortfolioAnnualized long2

let stdDevLongShort = stdDevPortfolioAnnualized longShort
let stdDevLongShort1 = stdDevPortfolioAnnualized longShort1
let stdDevLongShort2 = stdDevPortfolioAnnualized longShort2

let stdDevMarket = stdDevPortfolioAnnualized market
let stdDevMarket1 = stdDevPortfolioAnnualized market1
let stdDevMarket2 = stdDevPortfolioAnnualized market2

In [41]:
// for each portfolio stores sharpe ratio

let srLong = srPortfolio long
let srLong1 = srPortfolio long1
let srLong2 = srPortfolio long2

let srLongShort = srPortfolio longShort
let srLongShort1 = srPortfolio longShort1
let srLongShort2 = srPortfolio longShort2

let srMarket = srPortfolio market
let srMarket1 = srPortfolio market1
let srMarket2 = srPortfolio market2

In [42]:
// regressions of both the Long-Only and the Long-Short portfolios

let regLong = fitRegs (adaptRD long)
let regLong1 = fitRegs (adaptRD long1)
let regLong2 = fitRegs (adaptRD long2)

let regLongShort = fitRegs (adaptRD longShort)
let regLongShort1 = fitRegs (adaptRD longShort1)
let regLongShort2 = fitRegs (adaptRD longShort2)

In [43]:
// creates table with portfolio statistics

let header = 
    [ 
    "<b>Portfolio"      ; "<b>Avg. Excess Return"       ; "<b>Avg. Volatility"          ; "<b>Sharpe Ratio"         ; "<b>CAPM Alpha"                          ; "<b>CAPM Alpha t-stat"                        ; "<b>CAPM R-squared"                   ; "<b>CAPM IR"                          ; "<b>FF3 Alpha"                        ; "<b>FF3 Alpha t-stat"                     ; "<b>FF3 R-squared"                ; "<b>FF3 IR"                       
    ]

let rows = 
    [
     ["Long-only"       ; roundDisplay returnLong       ; roundDisplay stdDevLong       ; roundDisplay srLong       ; roundDisplay regLong.Alpha_CAPM          ; roundDisplay regLong.Alpha_TstatCAPM          ; roundDisplay regLong.R2_CAPM          ; roundDisplay regLong.IR_CAPM          ; roundDisplay regLong.Alpha_FF3        ; roundDisplay regLong.Alpha_TstatFF3       ; roundDisplay regLong.R2_FF3       ; roundDisplay regLong.IR_FF3       ]           
     ["Long-only 1st"   ; roundDisplay returnLong1      ; roundDisplay stdDevLong1      ; roundDisplay srLong1      ; roundDisplay regLong1.Alpha_CAPM         ; roundDisplay regLong1.Alpha_TstatCAPM         ; roundDisplay regLong1.R2_CAPM         ; roundDisplay regLong1.IR_CAPM         ; roundDisplay regLong1.Alpha_FF3       ; roundDisplay regLong1.Alpha_TstatFF3      ; roundDisplay regLong1.R2_FF3      ; roundDisplay regLong1.IR_FF3      ]   
     ["Long-only 2nd"   ; roundDisplay returnLong2      ; roundDisplay stdDevLong2      ; roundDisplay srLong2      ; roundDisplay regLong2.Alpha_CAPM         ; roundDisplay regLong2.Alpha_TstatCAPM         ; roundDisplay regLong2.R2_CAPM         ; roundDisplay regLong2.IR_CAPM         ; roundDisplay regLong2.Alpha_FF3       ; roundDisplay regLong2.Alpha_TstatFF3      ; roundDisplay regLong2.R2_FF3      ; roundDisplay regLong2.IR_FF3      ]
     ["Long-Short"      ; roundDisplay returnLongShort  ; roundDisplay stdDevLongShort  ; roundDisplay srLongShort  ; roundDisplay regLongShort.Alpha_CAPM     ; roundDisplay regLongShort.Alpha_TstatCAPM     ; roundDisplay regLongShort.R2_CAPM     ; roundDisplay regLongShort.IR_CAPM     ; roundDisplay regLongShort.Alpha_FF3   ; roundDisplay regLongShort.Alpha_TstatFF3  ; roundDisplay regLongShort.R2_FF3  ; roundDisplay regLongShort.IR_FF3  ] 
     ["Long-Short 1st"  ; roundDisplay returnLongShort1 ; roundDisplay stdDevLongShort1 ; roundDisplay srLongShort1 ; roundDisplay regLongShort1.Alpha_CAPM    ; roundDisplay regLongShort1.Alpha_TstatCAPM    ; roundDisplay regLongShort1.R2_CAPM    ; roundDisplay regLongShort1.IR_CAPM    ; roundDisplay regLongShort1.Alpha_FF3  ; roundDisplay regLongShort1.Alpha_TstatFF3 ; roundDisplay regLongShort1.R2_FF3 ; roundDisplay regLongShort1.IR_FF3 ] 
     ["Long-Short 2nd"  ; roundDisplay returnLongShort2 ; roundDisplay stdDevLongShort2 ; roundDisplay srLongShort2 ; roundDisplay regLongShort2.Alpha_CAPM    ; roundDisplay regLongShort2.Alpha_TstatCAPM    ; roundDisplay regLongShort2.R2_CAPM    ; roundDisplay regLongShort2.IR_CAPM    ; roundDisplay regLongShort2.Alpha_FF3  ; roundDisplay regLongShort2.Alpha_TstatFF3 ; roundDisplay regLongShort2.R2_FF3 ; roundDisplay regLongShort2.IR_FF3 ]
     ["Market"          ; roundDisplay returnMarket     ; roundDisplay stdDevMarket     ; roundDisplay srMarket     ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
     ["Market 1st"      ; roundDisplay returnMarket1    ; roundDisplay stdDevMarket1    ; roundDisplay srMarket1    ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
     ["Market 2nd"      ; roundDisplay returnMarket2    ; roundDisplay stdDevMarket2    ; roundDisplay srMarket2    ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
    ]
 

Chart.Table(header, rows) 
|> Chart.withSize (1550, 450) 
|> Chart.withTitle("Performance Measures")

<!-- Plotly chart will be drawn inside this DIV -->

### **2.2.** (Constant) Mean-Variance Optimization and Portfolio Diversification

In [44]:
// creates type for stock data
type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }

// tickers list for 60/40 portfolio
let tickers6040 = 
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

// tickers list including ff3 factors
let tickersFactors = 
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
        "VBR" // Vanguard Small-cap Value ETF
        "VUG" // Vanguard Growth ETF
    ]

// get tickers data
let tickPrices (tickers: string list)= 
    YahooFinance.History(
        tickers,
        startDate = DateTime(2000,1,1),
        interval = Interval.Monthly)

// store ticker prices for both portfolios
let tickPrices6040 = tickPrices tickers6040
let tickPricesFactors = tickPrices tickersFactors

// calculate returns from prices
let pricesToReturns (symbol, adjPrices: list<Quote>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })

// group returns by symbol
let tickReturns (tickersPrices: list<Quote>) =
    tickersPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns

// store returns for both portfolios
let tickReturns6040 = tickReturns tickPrices6040
let tickReturnsFactors = tickReturns tickPricesFactors

// get ff3 data in list format
let ff3 = French.getFF3 French.Frequency.Monthly |> Array.toList

// store rf rates
let rf = Map [ for x in ff3 do x.Date, x.Rf ]

// calculate excess returns for each of the downloaded tickers
let standardInvestmentsExcess (tickerReturns: StockData list) =
    let maxff3Date = ff3 |> List.map(fun x -> x.Date) |> List.max
    let minff3Date = ff3 |> List.map(fun x -> x.Date) |> List.min
    tickerReturns
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.filter(fun x -> x.Date >= minff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"no rf for {x.Date}"
        | Some rf -> { x with Return = x.Return - rf })

// store excess returns for each of the portfolios
let excReturns6040 = standardInvestmentsExcess tickReturns6040
let excReturnsFactors = standardInvestmentsExcess tickReturnsFactors

In [45]:
// adjust long-only to StockData type
let longAdjusted =
  long
  |> List.map(fun x ->
                { Symbol = "Long-Only"
                  Date = x.Month
                  Return = x.Return
                })

// adjust long-short to StockData type
let longShortAdjusted =
  longShort
  |> List.map(fun x ->
                { Symbol = "Long-Short"
                  Date = x.Month
                  Return = x.Return
                })

// join all data for Long portfolio w/ 60-40
let allStockDataLong6040 =
    List.concat [|excReturns6040; longAdjusted|]

// join all data for Long portfolio w/ Factors
let allStockDataLongFactors =
    List.concat [|excReturnsFactors; longAdjusted|]


// join all data for Long-Short portfolio w/ 60-40
let allStockDataLongShort6040 =
    List.concat [|excReturns6040; longShortAdjusted|]

// join all data for Long-Short portfolio w/ Factors
let allStockDataLongShortFactors =
    List.concat [|excReturnsFactors; longShortAdjusted|]

In [46]:
// necessary for dsharp
Formatter.Register<Tensor>(
    Action<_, _>(fun t (writer: IO.TextWriter) -> fprintfn writer "%120A" t),"text/plain")
Formatter.Register<Tensor>(
    Action<_, _>(fun t (writer: IO.TextWriter) -> fprintfn writer "%120A" t),"text/html")
Formatter.SetPreferredMimeTypesFor(typeof<Tensor>, "text/plain")

In [47]:
// store tickers for Long-Only 60-40 comparison
let tickersLong6040 = ["Long-Only"; "VTI"; "BND"]

// store tickers for Long-Only Factors comparison
let tickersLongFactors = ["Long-Only"; "VTI"; "BND"; "VBR"; "VUG"]         

// store tickers for Long-Short 60-40 comparison
let tickersLongShort6040 = ["Long-Short"; "VTI"; "BND"]

// store tickers for Long-Short Factors comparison
let tickersLongShortFactors = ["Long-Short"; "VTI"; "BND"; "VBR"; "VUG"]       

In [48]:
// function to calculate covariance between returns
let getCov xId yId (data: StockData list) =
    let xRet = 
        data
        |> List.filter(fun x -> x.Symbol = xId)
        |> List.map(fun x -> x.Date, x.Return)
        |> Map
    let yRet = 
        data
        |> List.filter(fun x -> x.Symbol = yId)
        |> List.map(fun x -> x.Date, x.Return)
        |> Map
    let overlappingDates =
        [ xRet.Keys |> set
          yRet.Keys |> set]
        |> Set.intersectMany
    [ for date in overlappingDates do xRet[date], yRet[date]]
    |> Seq.covOfPairs

// function to create covariances matrix
let covariances (ticks: string list) (data: StockData list) =
    [ for rowTick in ticks do 
        [ for colTick in ticks do
            getCov rowTick colTick data]]
    |> dsharp.tensor

// store covariances for all portfolios

let covLong6040 = covariances tickersLong6040 allStockDataLong6040
let covLongFactors = covariances tickersLongFactors allStockDataLongFactors

let covLongShort6040 = covariances tickersLongShort6040 allStockDataLongShort6040
let covLongShortFactors = covariances tickersLongShortFactors allStockDataLongShortFactors

In [49]:
// function to calculate mean returns by ticker
let meansByTick (data: StockData list) =
    data
    |> List.groupBy (fun x -> x.Symbol)
    |> List.map (fun (sym, xs) ->
        let symAvg = xs |> List.averageBy (fun x -> x.Return)
        sym, symAvg)
    |> Map

// get means by ticker to dsharp format
let means (tickers: string list) (data: StockData list) =
    // Make sure ticker avg returns are ordered by our ticker list
    [ for ticker in tickers do meansByTick(data)[ticker]]
    |> dsharp.tensor

// store means for all portfolios

let meansLong6040 = means tickersLong6040 allStockDataLong6040
let meansLongFactors = means tickersLongFactors allStockDataLongFactors

let meansLongShort6040 = means tickersLongShort6040 allStockDataLongShort6040
let meansLongShortFactors = means tickersLongShortFactors allStockDataLongShortFactors

In [50]:
// function to calculate optimal weights
let calcWeights (covars: Tensor) (means: Tensor) (tickers: string list) =
    let w' = dsharp.solve(covars,means)
    let w = w' / w'.sum()
    let weights =
        Seq.zip tickers (w.toArray1D<float>())
        |> Map.ofSeq
    weights  

// mean-variance optimal weights for the Long-Only 60-40 portfolio
let weightsLong6040 = calcWeights covLong6040 meansLong6040 tickersLong6040

// mean-variance optimal weights for the Long-Only Factors portfolio
let weightsLongFactors = calcWeights covLongFactors meansLongFactors tickersLongFactors

// mean-variance optimal weights for the Long-Short 60-40 portfolio
let weightsLongShort6040 = calcWeights covLongShort6040 meansLongShort6040 tickersLongShort6040

// mean-variance optimal weights for the Long-Only Factors portfolio
let weightsLongShortFactors = calcWeights covLongShortFactors meansLongShortFactors tickersLongShortFactors

// assign optimal weights for 60/40 portfolio
let weights6040 = Map [("VTI",0.6);("BND",0.4)]

In [51]:
// function to transform data in order to get complete data
let getCompleteData (tickers: string list) (data: StockData list) = 

    // sort data by date
    let stockDataByDate (data: StockData list) =
        data
        |> List.groupBy(fun x -> x.Date) // group all symbols on the same date together.
        |> List.sortBy (fun (dt, xs) -> dt) 

    // find the first array element where there are as many stocks as we have symbols
    let allAssetsStart (tickers: string list) (data: StockData list) =
        stockDataByDate data
        |> List.find(fun (month, stocks) -> stocks.Length = tickers.Length)
        |> fst

    // find the last array element where there are as many stocks as we have symbols
    let allAssetsEnd (tickers: string list) (data: StockData list) =
        stockDataByDate data
        |> List.findBack(fun (month, stocks) -> stocks.Length = tickers.Length)
        |> fst

    // get the data where all tickers are available
    let stockDataByDateComplete (tickers: string list) (data: StockData list) =
        stockDataByDate data
        |> List.filter(fun (date, stocks) -> 
            date >= allAssetsStart tickers data &&
            date <= allAssetsEnd tickers data)

    // check if we got all the data
    let checkOfCompleteData (tickers: string list) (data: StockData list) =
        stockDataByDateComplete tickers data
        |> List.map snd
        |> List.filter(fun x -> x.Length <> tickers.Length) // discard rows where we have all symbols.

    if not (List.isEmpty (checkOfCompleteData tickers data)) then 
            failwith "stockDataByDateComplete has months with missing stocks"
    
    // return final list
    stockDataByDateComplete tickers data

In [52]:
// get complete data for Long-Only 60-40 portfolio
let completeDataLong6040 = getCompleteData tickersLong6040 allStockDataLong6040

// get complete data for Long-Only Factors portfolio
let completeDataLongFactors = getCompleteData tickersLongFactors allStockDataLongFactors

// get complete data for Long-Short portfolio
let completeDataLongShort6040 = getCompleteData tickersLongShort6040 allStockDataLongShort6040

// get complete data for Long-Only Factors portfolio
let completeDataLongShortFactors = getCompleteData tickersLongShortFactors allStockDataLongShortFactors

In [53]:
// function to calculate monthly returns of portfolio according to given weighting scheme
let portfolioMonthReturn weights monthData =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum

In [54]:
// function to calculate returns for optimal mve weights
let listMveReturns (data: list<DateTime*list<StockData>>) (symbol: string) (weights: Map<string,float>) =
    data
    |> List.map(fun (date, obs) -> 
        { Symbol = symbol
          Date = date
          Return = portfolioMonthReturn weights obs })  

// calculate returns for each portfolio according to mve optimal weights
let retMveLong6040 = listMveReturns completeDataLong6040 "MVE-Long-6040" weightsLong6040
let retMveLongFactors = listMveReturns completeDataLongFactors "MVE-Long-Factors" weightsLongFactors
let retMveLongShort6040 = listMveReturns completeDataLongShort6040 "MVE-Long-Short-6040" weightsLongShort6040
let retMveLongShortFactors = listMveReturns completeDataLongShortFactors "MVE-Long-Short-Factors" weightsLongShortFactors
let ret6040 = listMveReturns completeDataLong6040 "60/40" weights6040

In [55]:
// adapt function to calculate cumulative returns for stock data type
let cumulateReturns (xs: StockData list) =
    let mutable cr = 1.0
    [ for x in xs do 
        cr <- cr * (1.0 + x.Return)
        { x with Return = cr } ]

// apply to all portfolios
let portMveLong6040Cumulative = retMveLong6040 |> cumulateReturns
let portMveLongFactorsCumulative = retMveLongFactors |> cumulateReturns
let portMveLongShort6040Cumulative = retMveLongShort6040 |> cumulateReturns
let portMveLongShortFactorsCumulative = retMveLongShortFactors |> cumulateReturns
let port6040Cumulative = ret6040 |> cumulateReturns

// adapt function to create charts for stock data type
let chartReturns (cumulativeReturns: StockData list) = 
    cumulativeReturns
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line

// create charts for each portfolio
let chartLong6040 = chartReturns portMveLong6040Cumulative |> Chart.withTraceInfo(Name="MVE Long-Only 60/40")
let chartLongFactors = chartReturns portMveLongFactorsCumulative |> Chart.withTraceInfo(Name="MVE Long-Only Factors")
let chartLongShort6040 = chartReturns portMveLongShort6040Cumulative |> Chart.withTraceInfo(Name="MVE Long-Short 60/40")
let chartLongShortFactors = chartReturns portMveLongShortFactorsCumulative |> Chart.withTraceInfo(Name="MVE Long-Short Factors")
let chart6040 = chartReturns port6040Cumulative |> Chart.withTraceInfo(Name="60/40")

// combine charts to single plot
let chartCombined =
    [ chartLong6040; chartLongFactors; chartLongShort6040; chartLongShortFactors; chart6040 ]
    |> Chart.combine

chartCombined

<!-- Plotly chart will be drawn inside this DIV -->

In [56]:
// adapt standard deviation and leverage to 10% volatility functions to the stock data type

let stdDevPortfolioAnnualized (xs: StockData list) =
    xs
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

let vol10 (xs: StockData list) =
    xs
    |> List.map(fun x ->
        let w = 0.1 / stdDevPortfolioAnnualized xs
        { Symbol = x.Symbol
          Date = x.Date
          Return = x.Return * w
        })

// apply to all portfolios
let portMveLong6040Vol = retMveLong6040 |> vol10 |> cumulateReturns
let portMveLongFactorsVol = retMveLongFactors |> vol10 |> cumulateReturns
let portMveLongShort6040Vol = retMveLongShort6040 |> vol10 |> cumulateReturns
let portMveLongShortFactorsVol = retMveLongShortFactors |> vol10 |> cumulateReturns
let port6040Vol = ret6040 |> vol10 |> cumulateReturns

// create charts for each portfolio
let chartLong6040Vol = chartReturns portMveLong6040Vol |> Chart.withTraceInfo(Name="MVE Long-Only 60/40")
let chartLongFactorsVol = chartReturns portMveLongFactorsVol |> Chart.withTraceInfo(Name="MVE Long-Only Factors")
let chartLongShort6040Vol = chartReturns portMveLongShort6040Vol |> Chart.withTraceInfo(Name="MVE Long-Short 60/40")
let chartLongShortFactorsVol = chartReturns portMveLongShortFactorsVol |> Chart.withTraceInfo(Name="MVE Long-Short Factors")
let chart6040Vol = chartReturns port6040Vol |> Chart.withTraceInfo(Name="60/40")

// combine charts to single plot
let chartCombinedVol =
    [ chartLong6040Vol; chartLongFactorsVol; chartLongShort6040Vol; chartLongShortFactorsVol; chart6040Vol ]
    |> Chart.combine

chartCombinedVol

<!-- Plotly chart will be drawn inside this DIV -->

In [57]:
// adapt average return and sharpe ratio functions to the stock data type

let returnsPortfolioAnnualized (xs: StockData list) =
    xs
    |> Seq.averageBy (fun x -> x.Return)
    |> annualizeMonthlyReturns

let srPortfolio (xs: StockData list) = 
    (returnsPortfolioAnnualized xs) / (stdDevPortfolioAnnualized xs)

In [58]:
// calculate average return, volatility and sharpe ratio for all portfolios

let returnMveLong6040 = returnsPortfolioAnnualized retMveLong6040
let returnMveLongFactors = returnsPortfolioAnnualized retMveLongFactors
let returnMveLongShort6040 = returnsPortfolioAnnualized retMveLongShort6040
let returnMveLongShortFactors = returnsPortfolioAnnualized retMveLongShortFactors
let return6040 = returnsPortfolioAnnualized ret6040

let stdDevMveLong6040 = stdDevPortfolioAnnualized retMveLong6040
let stdDevMveLongFactors = stdDevPortfolioAnnualized retMveLongFactors
let stdDevMveLongShort6040 = stdDevPortfolioAnnualized retMveLongShort6040
let stdDevMveLongShortFactors = stdDevPortfolioAnnualized retMveLongShortFactors
let stdDev6040 = stdDevPortfolioAnnualized ret6040

let srMveLong6040 = srPortfolio retMveLong6040
let srMveLongFactors = srPortfolio retMveLongFactors
let srMveLongShort6040 = srPortfolio retMveLongShort6040
let srMveLongShortFactors = srPortfolio retMveLongShortFactors
let sr6040 = srPortfolio ret6040

In [59]:
let header = 
    [ 
    "<b>Portfolio"          ; "<b>Avg. Excess Return"                   ; "<b>Avg. Volatility"                      ; "<b>Sharpe Ratio"                                
    ]

let rows = 
    [
     ["Long-Only 60/40"     ; roundDisplay returnMveLong6040            ; roundDisplay stdDevMveLong6040            ; roundDisplay srMveLong6040            ]           
     ["Long-Only Factors"   ; roundDisplay returnMveLongFactors         ; roundDisplay stdDevMveLongFactors         ; roundDisplay srMveLongFactors         ]   
     ["Long-Short 60/40"    ; roundDisplay returnMveLongShort6040       ; roundDisplay stdDevMveLongShort6040       ; roundDisplay srMveLongShort6040       ] 
     ["Long-Short Factors"  ; roundDisplay returnMveLongShortFactors    ; roundDisplay stdDevMveLongShortFactors    ; roundDisplay srMveLongShortFactors    ] 
     ["60/40"               ; roundDisplay return6040                   ; roundDisplay stdDev6040                   ; roundDisplay sr6040                   ] 
    ]
 

Chart.Table(header, rows) 
|> Chart.withSize (1000, 450) 
|> Chart.withTitle("Performance Measures")

<!-- Plotly chart will be drawn inside this DIV -->

In [60]:
// Observe optimal weights for each portfolio
weightsLong6040

key,value
BND,0.8475341796875
Long-Only,0.663883626461029
VTI,-0.5114177465438843


In [61]:
weightsLongFactors

key,value
BND,0.8352622985839844
Long-Only,0.7396761178970337
VBR,0.06738697737455368
VTI,-0.9581042528152466
VUG,0.3157788813114166


In [62]:
weightsLongShort6040

key,value
BND,0.5279969573020935
Long-Short,0.26132524013519287
VTI,0.21067778766155243


In [63]:
weightsLongShortFactors

key,value
BND,0.5088613033294678
Long-Short,0.27089381217956543
VBR,0.12420604377985
VTI,-0.22570571303367615
VUG,0.32174450159072876


### **2.3.** (Moving) Mean-Variance Optimization and Portfolio Diversification

In [64]:
// function to add one year to any date
let addOneYear (dt: DateTime) =
    dt.AddYears(1)

// creates type store signal
type StoreSignal =
    { Date: DateTime
      Weights: Map<string,float> }

// function to calculate both moving mve returns and portfolio based on past observations
let createMovingMvePortfolio (allData: list<StockData>) (completeData: list<DateTime * list<StockData>>) (tickers: list<string>) (label: string) = 

    // gathers a list with all years after 2010 (years where we will test the strategy)
    let dates = 
        allData
        |> List.filter(fun x -> x.Date >= DateTime(2010,1,1))
        |> List.map (fun x -> x.Date.Year)
        |> List.distinct
        |> List.sort

    // creates mutable lists to store both signals and returns
    let mutable storeSignals: StoreSignal list = List.empty
    let mutable storeReturns = List.empty

    // get dates where data is complete to use
    let minDateComplete = completeData |> List.map(fun (dt, obs) -> dt) |> List.min
    let maxDateComplete = completeData |> List.map(fun (dt, obs) -> dt) |> List.max

    // for every year in the test set
    for date = 0 to dates.Length-1 do

        // create training set until starting date of test set
        let train = 
            allData
            |> List.sortBy(fun x -> x.Date)
            |> List.filter(fun x -> x.Date < DateTime(dates[date],1,1))

        // gather dates where the year of the test set can be used (verify if they have all tickers)
        let startDataTest = if DateTime(dates[date],1,1) > minDateComplete then DateTime(dates[date],1,1) else minDateComplete
        let endDataTest = if addOneYear(DateTime(dates[date],1,1)) < maxDateComplete then addOneYear(DateTime(dates[date],1,1)) else maxDateComplete

        // create test set
        let test = 
            allData
            |> List.sortBy(fun x -> x.Date)
            |> List.filter(fun x -> x.Date >= startDataTest)
            |> List.filter(fun x -> x.Date < endDataTest)

        // calculate optimal weights according to the training set
        let covTrain = covariances tickers train
        let meansTrain = means tickers train
        let weightsTrain = calcWeights covTrain meansTrain tickers

        // apply optimal weights in training set to calculate returns in the test set
        let completeTest = test |> List.groupBy(fun x -> x.Date) |> List.sortBy (fun (dt, xs) -> dt)
        let testReturns = listMveReturns completeTest label weightsTrain

        // create list with test year and weights used
        let storeSignalDate =
            [{ Date = startDataTest
               Weights = weightsTrain }]

        // store both returns and weights of test
        storeReturns <- storeReturns @ testReturns
        storeSignals <- storeSignals @ storeSignalDate

    // return all returns and all weights
    storeReturns, storeSignals

In [65]:
// store returns for both long-short and long-only for Fama-French factors portfolios
let retsMovingMveLS, weightsMovingMveLS = createMovingMvePortfolio allStockDataLongShortFactors completeDataLongShortFactors tickersLongShortFactors "moving-MVE-LS"
let retsMovingMveLO, weightsMovingMveLO = createMovingMvePortfolio allStockDataLongFactors completeDataLongFactors tickersLongFactors "moving-MVE-LO"

In [66]:
// calculate cumulative returns for all portfolios and chart them

let portMovingMveFactorsLS = retsMovingMveLS |> cumulateReturns
let chartMovingMveFactorsLS = chartReturns portMovingMveFactorsLS |> Chart.withTraceInfo(Name="Moving MVE Factors Long-Short")

let portMovingMveFactorsLO = retsMovingMveLO |> cumulateReturns
let chartMovingMveFactorsLO = chartReturns portMovingMveFactorsLO |> Chart.withTraceInfo(Name="Moving MVE Factors Long-Only")

let portMveLongShortFactorsCumulativeAdapt = retMveLongShortFactors |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)) |> cumulateReturns
let port6040CumulativeAdapt = ret6040 |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)) |> cumulateReturns

let chartLongShortFactorsAdapt = chartReturns portMveLongShortFactorsCumulativeAdapt |> Chart.withTraceInfo(Name="Constant MVE Long-Short Factors")
let chart6040Adapt = chartReturns port6040CumulativeAdapt |> Chart.withTraceInfo(Name="60/40")

// combine charts to single plot
let chartMovingCombined =
    [ chartMovingMveFactorsLS; chartMovingMveFactorsLO; chartLongShortFactorsAdapt; chart6040Adapt ]
    |> Chart.combine

chartMovingCombined

<!-- Plotly chart will be drawn inside this DIV -->

In [67]:
// calculate cumulative returns for leveraged portfolios targetting 10% volatility

let portMovingMveFactorsLSVol = retsMovingMveLS |> vol10 |> cumulateReturns
let portMovingMveFactorsLOVol = retsMovingMveLO |> vol10 |> cumulateReturns
let portMveLongShortFactorsVolAdapt = retMveLongShortFactors |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)) |> vol10 |> cumulateReturns
let port6040Vol = ret6040 |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)) |> vol10 |> cumulateReturns

let chartMovingMveFactorsLSVol = chartReturns portMovingMveFactorsLSVol |> Chart.withTraceInfo(Name="Moving MVE Factors Long-Short")
let chartMovingMveFactorsLOVol = chartReturns portMovingMveFactorsLOVol |> Chart.withTraceInfo(Name="Moving MVE Factors Long-Only")
let chartLongShortFactorsAdaptVol = chartReturns portMveLongShortFactorsVolAdapt |> Chart.withTraceInfo(Name="Constant MVE Long-Short Factors")
let chart6040AdaptVol = chartReturns port6040Vol |> Chart.withTraceInfo(Name="60/40")

// combine charts to single plot
let chartMovingCombinedVol =
    [ chartMovingMveFactorsLSVol; chartMovingMveFactorsLOVol; chartLongShortFactorsAdaptVol; chart6040AdaptVol ]
    |> Chart.combine

chartMovingCombinedVol

<!-- Plotly chart will be drawn inside this DIV -->

In [68]:
// calculate average return, volatility and sharpe ratio for all portfolios

let returnMovingMveLS = returnsPortfolioAnnualized retsMovingMveLS
let returnMovingMveLO = returnsPortfolioAnnualized retsMovingMveLO
let returnMveLongShortFactors = returnsPortfolioAnnualized (retMveLongShortFactors |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)))
let return6040 = returnsPortfolioAnnualized (ret6040 |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)))

let stdDevMovingMveLS = stdDevPortfolioAnnualized retsMovingMveLS
let stdDevMovingMveLO = stdDevPortfolioAnnualized retsMovingMveLO
let stdDevMveLongShortFactors = stdDevPortfolioAnnualized (retMveLongShortFactors |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)))
let stdDev6040 = stdDevPortfolioAnnualized ret6040

let srMovingMveLS = srPortfolio retsMovingMveLS
let srMovingMveLO = srPortfolio retsMovingMveLO
let srMveLongShortFactors = srPortfolio (retMveLongShortFactors |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)))
let sr6040 = srPortfolio (ret6040 |> List.filter (fun x -> x.Date >= DateTime(2010,1,1)))

In [69]:
let header = 
    [ 
    "<b>Portfolio"                      ; "<b>Avg. Excess Return"                   ; "<b>Avg. Volatility"                      ; "<b>Sharpe Ratio"                                
    ]

let rows = 
    [
     ["Moving MVE Long-Short Factors"   ; roundDisplay returnMovingMveLS            ; roundDisplay stdDevMovingMveLS            ; roundDisplay srMovingMveLS            ]   
     ["Moving MVE Long-Only Factors"    ; roundDisplay returnMovingMveLO            ; roundDisplay stdDevMovingMveLO            ; roundDisplay srMovingMveLO            ] 
     ["Constant MVE Long-Short Factors" ; roundDisplay returnMveLongShortFactors    ; roundDisplay stdDevMveLongShortFactors    ; roundDisplay srMveLongShortFactors    ] 
     ["60/40"                           ; roundDisplay return6040                   ; roundDisplay stdDev6040                   ; roundDisplay sr6040                   ] 
    ]
 

Chart.Table(header, rows) 
|> Chart.withSize (1200, 450) 
|> Chart.withTitle("Performance Measures")

<!-- Plotly chart will be drawn inside this DIV -->

In [110]:
// Observe optimal weights for long-short portfolio
let getTickerWeightPlot (weightsList: list<StoreSignal>) (ticker: string) =
    weightsList
    |> List.map(fun obs ->
        obs.Date, obs.Weights[ticker])
    |> Chart.Line

In [112]:
[ getTickerWeightPlot (weightsMovingMveLS) ("Long-Short") |> Chart.withTraceInfo(Name="Long-Short")
  getTickerWeightPlot (weightsMovingMveLS) ("VTI") |> Chart.withTraceInfo(Name="Equity ETF")
  getTickerWeightPlot (weightsMovingMveLS) ("BND") |> Chart.withTraceInfo(Name="Bond ETF")
  getTickerWeightPlot (weightsMovingMveLS) ("VBR") |> Chart.withTraceInfo(Name="Size ETF")
  getTickerWeightPlot (weightsMovingMveLS) ("VUG") |> Chart.withTraceInfo(Name="Growth ETF")]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->